<a href="https://colab.research.google.com/github/Abdullahaml1/deep-learning/blob/main/text-summerization-project/cnnNews_dataset_prepration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.9 MB/s 
     |████████████████████████████████| 346 kB 64.0 MB/s 
     |████████████████████████████████| 6.6 MB 29.8 MB/s 
     |████████████████████████████████| 86 kB 4.4 MB/s 
     |████████████████████████████████| 596 kB 63.5 MB/s 
     |████████████████████████████████| 212 kB 57.0 MB/s 
     |████████████████████████████████| 86 kB 3.6 MB/s 
     |████████████████████████████████| 140 kB 57.4 MB/s 
     |████████████████████████████████| 1.1 MB 52.6 MB/s 
     |████████████████████████████████| 127 kB 72.2 MB/s 
     |████████████████████████████████| 94 kB 4.5 MB/s 
     |████████████████████████████████| 144 kB 74.4 MB/s 
     |████████████████████████████████| 271 kB 70.9 MB/s 
     |████████████████████████████████| 112 kB 77.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling

# [cnn\dailymail dataset](https://huggingface.co/datasets/cnn_dailymail)

In [ ]:
from datasets import load_dataset

dataset = load_dataset('cnn_dailymail', '3.0.0')

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset['train'][0]

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

# splition sentences using Stavford core NLP

In [ ]:
! pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 574 kB 5.3 MB/s 
     |████████████████████████████████| 175 kB 50.6 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=7688c94910c3395e8e91a2e56ee1d7cfd93a29ee40ecc9f2f763391483e3ff0a
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


# breaking articel into sentences

In [ ]:
import stanza
stanza.download('en') # download English model
nlp = stanza.Pipeline(processors='tokenize', lang='en') # initialize English neural pipeline

2022-06-07 13:19:59 INFO: Downloading default packages for language: en (English)...


2022-06-07 13:20:10 INFO: Finished downloading models and saved to /root/stanza_resources.


2022-06-07 13:20:10 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2022-06-07 13:20:10 INFO: Use device: gpu
2022-06-07 13:20:10 INFO: Loading: tokenize
2022-06-07 13:20:24 INFO: Done loading processors!


In [ ]:
# doc = nlp("Barack Obama was! born in Hawaii.") # run annotation over a sentence


def split_article_to_sentences_list(article):
  sent_list = []
  doc = nlp(article) # run annotation over a sentence
  for i, sentence in enumerate(doc.sentences):
      sent = ' '.join(word.text for word in sentence.words)
      sent_list.append(sent)
  return sent_list

split_article_to_sentences_list("Barack Obama was! born in Hawaii.")



['Barack Obama was !', 'born in Hawaii .']

In [ ]:
from tqdm import tqdm 
def split_dataset_sentences(dataset, key1, key2, max_lenth=1000):
  out_dataset = []
  for i in tqdm(range(max_lenth)):
    dic = {key1: split_article_to_sentences_list(dataset[i][key1]),
          key2:split_article_to_sentences_list(dataset[i][key2])}
    out_dataset.append(dic)
  
  return out_dataset

trainset = dataset['train']
trainset_splited = split_dataset_sentences(trainset, 'article', 'highlights', max_lenth=100)


100%|██████████| 100/100 [00:16<00:00,  5.99it/s]


In [ ]:
trainset_splited[0]['article']


["LONDON , England ( Reuters ) -- Harry Potter star Daniel Radcliffe gains access to a reported £ 20 million ( $ 41.1 million ) fortune as he turns 18 on Monday , but he insists the money wo n't cast a spell on him .",
 'Daniel Radcliffe as Harry Potter in " Harry Potter and the Order of the Phoenix " To the disappointment of gossip columnists around the world , the young actor says he has no plans to fritter his cash away on fast cars , drink and celebrity parties .',
 '" I do n\'t plan to be one of those people who , as soon as they turn 18 , suddenly buy themselves a massive sports car collection or something similar , " he told an Australian interviewer earlier this month .',
 '" I do n\'t think I \'ll be particularly extravagant .',
 '" The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs . "',
 'At 18 , Radcliffe will be able to gamble in a casino , buy a drink in a pub or see the horror film " Hostel : Part II , " currently six places below his

# generating extractive summary with highest rough2 

In [ ]:
! pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from rouge import Rouge 

hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to he    lp students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news"

reference = "this page includes the show transcript use the transcript to help students with reading comprehension and     vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teac    her or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests     students ' knowledge of even ts in the news"

rouge = Rouge()
rouge.get_scores(hypothesis, reference)

[{'rouge-1': {'f': 0.49411764217577864,
   'p': 0.5833333333333334,
   'r': 0.42857142857142855},
  'rouge-2': {'f': 0.23423422957552154,
   'p': 0.3170731707317073,
   'r': 0.18571428571428572},
  'rouge-l': {'f': 0.44705881864636676,
   'p': 0.5277777777777778,
   'r': 0.3877551020408163}}]

In [ ]:
import numpy as np

def get_sentences_with_highest_rouge2(article_list, summary_list):
  labels = np.zeros(len(summary_list)
  # # for sum_i in tqdm(range(len(summary_list))):
  # #   max_score=0
  # #   max_art_i=0
  # #   current_score=0
  # #   for art_i in range(len(article_list)):

  # #     current_score= rouge.get_scores(article_list[art_i], summary_list[sum_i])[0]['rouge-2']['f']
  # #     # print(sum_i, art_i, current_score)
  # #     if current_score > max_score:
  # #       max_score = current_score
  # #       max_art_i = art_i

  # #   labels[sum_i] = max_art_i

  # return labels

# get_sentences_with_highest_rouge2(trainset_splited[0]['article'], trainset_splited[0]['highlights'])

SyntaxError: ignored

In [ ]:
import numpy as np

def get_sentences_with_highest_rouge2(article_list, summary_list):
  labels = np.zeros((len(summary_list))
  # # for sum_i in tqdm(range(len(summary_list))):
  # #   max_score=0
  # #   max_art_i=0
  # #   current_score=0
  # #   for art_i in range(len(article_list)):

  # #     current_score= rouge.get_scores(article_list[art_i], summary_list[sum_i])[0]['rouge-2']['f']
  # #     # print(sum_i, art_i, current_score)
  # #     if current_score > max_score:
  # #       max_score = current_score
  # #       max_art_i = art_i

  # #   labels[sum_i] = max_art_i

  # return labels

# get_sentences_with_highest_rouge2(trainset_splited[0]['article'], trainset_splited[0]['highlights'])
np.zeros(3)

SyntaxError: ignored

In [ ]:


def convert_summary_to_labels(dataset, article_key, highlights_key):
  out_data = []
  for i in range(len(dataset)):
    dic = {'labels': get_sentences_with_highest_rouge2(article_list=dataset[i][article_key]
                                                       , summary_list=dataset[i][highlights_key]),
            'article': dataset[i][article_key]}
    out_data.append(dic)

  return out_data

convert_summary_to_labels(trainset_splited, 'article', 'highlights')[0]

100%|██████████| 25/25 [00:00<00:00, 1510.09it/s]

100%|██████████| 25/25 [00:00<00:00, 1646.01it/s]

100%|██████████| 47/47 [00:00<00:00, 1577.22it/s]

100%|██████████| 51/51 [00:00<00:00, 3277.40it/s]

100%|██████████| 51/51 [00:00<00:00, 2288.78it/s]

100%|██████████| 25/25 [00:00<00:00, 2874.15it/s]

100%|██████████| 25/25 [00:00<00:00, 3057.16it/s]

100%|██████████| 45/45 [00:00<00:00, 2062.80it/s]

100%|██████████| 45/45 [00:00<00:00, 3662.65it/s]

100%|██████████| 45/45 [00:00<00:00, 2064.31it/s]

100%|██████████| 40/40 [00:00<00:00, 1969.64it/s]

100%|██████████| 40/40 [00:00<00:00, 1460.87it/s]

100%|██████████| 54/54 [00:00<00:00, 2432.24it/s]

100%|██████████| 54/54 [00:00<00:00, 2585.35it/s]

100%|██████████| 15/15 [00:00<00:00, 1303.12it/s]

100%|██████████| 15/15 [00:00<00:00, 1999.26it/s]

100%|██████████| 15/15 [00:00<00:00, 1910.61it/s]

100%|██████████| 15/15 [00:00<00:00, 4597.67it/s]

100%|██████████| 16/16 [00:00<00:00, 2192.31it/s]

100%|██████████| 16/16 [00:00<0

ValueError: ignored